In [25]:
import nltk.stem
import re
import numpy as np
import os
import nltk
import pandas as pd
from src.SemEvalData import SemEvalData
from src.JigsawData import JigsawData
from src.preprocessing import preprocess_bayes, getSpansByToxicWords, getToxicWordsBayes
import joblib
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from test_sentence import test_bayes

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/patrycja/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#stemmer = nltk.stem.SnowballStemmer('english')
lemmer = nltk.stem.WordNetLemmatizer()

In [4]:
## load data
train_data_semeval = SemEvalData()
train_data_semeval.load_data("data/tsd_train.csv")
train_df_preprocessed = train_data_semeval.preprocess()

In [5]:
train_df_preprocessed.head(25)

,spans,text,toxicity,toxic_words,original_text,sentences,toxicity_sentence
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",another violent and aggressive immigrant killi...,1,[violent and aggressive immigrant],Another violent and aggressive immigrant killi...,[another violent and aggressive immigrant kill...,[1.0]
1,"[33, 34, 35, 36, 37, 38, 39]","i am 56 years old, i am not your fucking junio...",1,[fucking],"I am 56 years old, I am not your fucking junio...","[i am 56 years old, i am not your fucking juni...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[0, 1, 2, 3]","damn, a whole family. sad indeed.",1,[damn],"Damn, a whole family. Sad indeed.","[damn, a whole family., sad indeed.]","[1.0, 0.0]"
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",what a knucklehead. how can anyone not know th...,1,[knucklehead],What a knucklehead. How can anyone not know th...,"[what a knucklehead., how can anyone not know ...","[1.0, 0.0, 0.0]"
4,"[32, 33, 34, 35, 36, 37, 38]",who do you think should do the killing? anyon...,1,[killing],"""who do you think should do the killing?""\n\nA...","[who do you think should do the killing?, anyo...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
5,[],"but, but, but, is not a defense. its not even...",0,[],"But, but, but, is NOT a defense. It's not eve...","[but, but, but, is not a defense., its not eve...","[0.0, 0.0, 0.0, 0.0, 0.0]"
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...","please people, stop using these silly, stupid ...",1,"[stupid, emoticons]","Please people, stop using these silly, stupid ...","[please people, stop using these silly, stupid...",[1.0]
7,"[0, 1, 2, 3]",dumb.,1,[dumb],Dumb.,[dumb.],[1.0]
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",obamacare is on its last gasping breaths. yo...,1,"[idiots, stupid]",Obamacare is on it's last gasping breaths. Y...,"[obamacare is on its last gasping breaths., yo...","[0.0, 1.0, 0.0, 0.0]"
9,"[32, 33, 34, 35, 36, 37, 38, 39]",crooked trump = guilty as hell. pathetic,1,[pathetic],CROOKED Trump = GUILTY as hell.\npathetic,"[crooked trump = guilty as hell., pathetic]","[0.0, 1.0]"


In [6]:
train_data = {
    'sentence':  train_df_preprocessed.sentences.sum(),
    'toxicity_sentence': train_df_preprocessed.toxicity_sentence.sum()
        }

train_df = pd.DataFrame (train_data, columns = ['sentence','toxicity_sentence'])

In [7]:

train_df['sentence'] = train_df.apply(lambda row: preprocess_bayes(row.sentence), axis=1)
print(train_df.head(5))

                                            sentence  toxicity_sentence
0  another violent and aggressive immigrant killi...                1.0
1   i am  years old i am not your fucking junior pal                1.0
2                 what you are saying makes no sense                0.0
3            i dont know what you are basing this on                0.0
4  the cheap black market crap is still coming up...                0.0


In [11]:
len(train_df[train_df["toxicity_sentence"] == 0])

13755

In [12]:
len(train_df[train_df["toxicity_sentence"] == 1])

9009

In [13]:
def stemming(text):
    # Stem words
    #data[i] = ' '.join([stemmer.stem(word) for word in data[i].split()])
    data = ' '.join([lemmer.lemmatize(word) for word in text.split()])
    return data


In [14]:
# Perform a grid search to find the best hyperparameters
def grid_search(train):
    # Create a pipeline
    clf_pipeline = sklearn.pipeline.Pipeline([
        ('v', CountVectorizer(strip_accents='ascii', stop_words='english')),
        ('t', TfidfTransformer()), 
        ('c', sklearn.naive_bayes.MultinomialNB(fit_prior=True, class_prior=None))
        ])
    # Set parameters (name in pipeline + name of parameter)
    parameters = { 
        'v__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)], 
        'v__lowercase': (True, False), 
        't__use_idf': (True, False), 
        'c__alpha': (0.3, 0.6, 1.0) }
    # Create a grid search classifier
    gs_classifier = sklearn.model_selection.GridSearchCV(clf_pipeline, parameters, cv=5, iid=False, n_jobs=2, scoring='accuracy', verbose=1)
    
    # Start a search (Warning: takes a long time if the whole dataset is used)
    # Slice: (train.data[:4000], train.target[:4000])
    gs_classifier = gs_classifier.fit(train.data, train.target)
    # Print results
    print('---- Results ----')
    print('Best score: ' + str(gs_classifier.best_score_))
    for name in sorted(parameters.keys()):
        print('{0}: {1}'.format(name, gs_classifier.best_params_[name]))

In [15]:
train_df['sentence'] = train_df.apply(lambda row: stemming(row.sentence), axis=1)

In [16]:
print(train_df.head())

                                            sentence  toxicity_sentence
0  another violent and aggressive immigrant killi...                1.0
1     i am year old i am not your fucking junior pal                1.0
2                  what you are saying make no sense                0.0
3            i dont know what you are basing this on                0.0
4  the cheap black market crap is still coming up...                0.0


In [17]:
# Train and evaluate a model
def train_and_evaluate(train):
    
    # Convert to bag of words
    count_vect = CountVectorizer(strip_accents='ascii', stop_words='english', lowercase=True, ngram_range=(1,1))
    X = count_vect.fit_transform(train['sentence'])
    # Convert from occurrences to frequencies
    # Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
    # To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.
    transformer = TfidfTransformer()
    X = transformer.fit_transform(X)
    # Create a model
    # get the first vector out (for the first document) 
    first_vector_tfidfvectorizer=X[0] 
 
    # place tf-idf values in a pandas data frame 
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=count_vect.get_feature_names(), columns=["tfidf"]) 
    df.sort_values(by=["tfidf"],ascending=True)
    print(df.head)
    model = sklearn.naive_bayes.MultinomialNB(alpha=0.3, fit_prior=True, class_prior=None)
    # Train the model
    model.fit(X, train['toxicity_sentence'])
    # Save models
    joblib.dump(count_vect, 'vectorizer.jbl')
    joblib.dump(transformer, 'transformer.jbl')
    joblib.dump(model, 'model.jbl')
    # Evaluate on training data
    print('-- Training data --')
    predictions = model.predict(X)
    print('here',model.coef_, len(count_vect.get_feature_names()), predictions)
#     neg_class_prob_sorted = model.feature_log_prob_[0, :].argsort()
#     pos_class_prob_sorted = model.feature_log_prob_[1, :].argsort()

#     print(np.take(count_vect.get_feature_names(), neg_class_prob_sorted[:10]))
#     print(np.take(count_vect.get_feature_names(), pos_class_prob_sorted[:10]))
    print(predictions,X, train['sentence'])
    accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
    print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
    print('Classification Report:')
    print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))
    print('')
#     Evaluate with 10-fold CV
    print('-- 10-fold CV --')
    predictions = sklearn.model_selection.cross_val_predict(model, X, train['toxicity_sentence'], cv=10)
    accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
    print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
    print('Classification Report:')
    print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))
    return model


In [18]:

train_and_evaluate(train_df)

<bound method NDFrame.head of          tfidf
aa         0.0
aaa        0.0
aahhh      0.0
aapoor     0.0
abandon    0.0
...        ...
zuptoid    0.0
zuri       0.0
zwei       0.0
zz         0.0
zzzzzz     0.0

[17812 rows x 1 columns]>
-- Training data --
here [[ -8.95407603 -10.42086336 -10.3399637  ... -11.38049662 -11.38049662
  -11.38049662]] 17812 [1. 1. 0. ... 0. 0. 1.]
[1. 1. 0. ... 0. 0. 1.]   (0, 17025)	0.3612465882102366
  (0, 13766)	0.4250020107598058
  (0, 8722)	0.2989512824534744
  (0, 8171)	0.349476259432824
  (0, 8079)	0.3323369916456108
  (0, 7760)	0.3248117045584413
  (0, 2604)	0.3030879853045209
  (0, 320)	0.41177156518958624
  (1, 17689)	0.31672999210173375
  (1, 11354)	0.4934807414263961
  (1, 10986)	0.3800480289834355
  (1, 8568)	0.5424937095001565
  (1, 6174)	0.4662863291616598
  (2, 14090)	0.658914520992524
  (2, 13808)	0.5930647728437727
  (2, 9514)	0.46271571103319686
  (3, 8797)	0.3879256621830136
  (3, 4524)	0.35660724746419836
  (3, 1290)	0.8499087902097956

MultinomialNB(alpha=0.3)

In [19]:
##load from file
count_vect = joblib.load('vectorizer.jbl')
transformer = joblib.load('transformer.jbl')
model = joblib.load('model.jbl')

In [22]:
samples_to_test = train_df.head(19)
for sample in samples_to_test.itertuples():
    print(sample)
    x = count_vect.transform([sample.sentence])
    y = model.predict(x)
    if y == 1.0:
        toxic = getToxicWordsBayes(count_vect, x, 0.5)
        spans = getSpansByToxicWords(toxic,sample.sentence)

Pandas(Index=0, sentence='another violent and aggressive immigrant killing a innocent and intelligent u citizen sarcasm', toxicity_sentence=1.0)
Pandas(Index=1, sentence='i am year old i am not your fucking junior pal', toxicity_sentence=1.0)
Pandas(Index=2, sentence='what you are saying make no sense', toxicity_sentence=0.0)
Pandas(Index=3, sentence='i dont know what you are basing this on', toxicity_sentence=0.0)
Pandas(Index=4, sentence='the cheap black market crap is still coming up from mexican national and the good stuff from bc and humbolt', toxicity_sentence=0.0)
Pandas(Index=5, sentence='the i corridor is still just a busy and they are still making a ton of money', toxicity_sentence=0.0)
Pandas(Index=6, sentence='nothing ha really changed except that the legal market ha made it easy for anyone of legal age to buy it', toxicity_sentence=0.0)
Pandas(Index=7, sentence='and the legal market is controlled by corporate grower with million dollar facility or a coop of smaller grower'

In [34]:
test_data_semeval = SemEvalData()
test_data_semeval.load_data("data/tsd_trial.csv")
test_df_preprocessed = test_data_semeval.preprocess()

In [37]:
test_df_preprocessed["predicted_span"]=[test_bayes(sentences[0])for sentences in test_df_preprocessed["sentences"] ]

In [38]:
test_df_preprocessed

,spans,text,toxicity,toxic_words,original_text,sentences,toxicity_sentence,predicted_span
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",because hes a moron and a bigot. its not any m...,1,"[moron, bigot]",Because he's a moron and a bigot. It's not any...,"[because hes a moron and a bigot., its not any...","[1.0, 0.0]","[26, 27, 28, 29, 30, 14, 15, 16, 17, 18]"
1,"[29, 30, 31, 32, 33, 34]",how about we stop protecting idiots and let na...,1,[idiots],How about we stop protecting idiots and let na...,[how about we stop protecting idiots and let n...,"[1.0, 0.0]","[51, 52, 53, 60, 61, 62, 63, 64, 65, 74, 75, 7..."
2,"[166, 167, 168, 169, 170, 171]","if people were smart, they would boycott th...",1,[idiots],"If people were smart, they would Boycott th...","[if people were smart, they would boycott t...",[1.0],[]
3,"[87, 88, 89, 90, 91, 92]",trump claimed that russia will never invade th...,1,[stupid],Trump Claimed that Russia will never invade th...,[trump claimed that russia will never invade t...,[1.0],[]
4,[],as long as your willing to pay a lot more for ...,0,[],As long as your willing to pay a lot more for ...,[as long as your willing to pay a lot more for...,"[0.0, 0.0, 0.0]",[]
...,...,...,...,...,...,...,...,...
685,"[129, 130, 131, 132, 133, 134]",but ... trumps not bluffing. hes prepared to g...,1,[stupid],But ... Trump's not bluffing. He's prepared to...,"[but ... trumps not bluffing., hes prepared to...","[0.0, 0.0, 0.0, 1.0, 0.0]",[]
686,"[126, 127, 128, 129, 130, 131]",cant believe the limited knowledge of this art...,1,[stupid],Can't believe the limited knowledge of this Ar...,[cant believe the limited knowledge of this ar...,"[0.0, 1.0]","[43, 44, 45, 46, 47, 48, 49, 5, 6, 7, 8, 9, 10..."
687,"[24, 25, 26, 27, 28, 29]",i think it conservative idiots who cannot reac...,1,[idiots],I think it conservative idiots who cannot reac...,[i think it conservative idiots who cannot rea...,[1.0],"[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2..."
688,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",youre an id*ot...go away.,1,[youre an id*ot],You're an id*ot...Go away.,[youre an id*ot...go away.],[1.0],[]


In [40]:
test_df_preprocessed["Pscore"] = [ 1 if (len(s) == 0 and len(ps) == 0) 
                             else 0 if len(ps) == 0 
                             else len( set(s).intersection(set(ps) ))/ len(set(ps))  for s, ps in zip(test_df_preprocessed["spans"],test_df_preprocessed["predicted_span"]) ]

In [41]:
test_df_preprocessed["Rscore"] = [ 1 if (len(s) == 0 and len(ps) == 0) 
                             else 0 if len(s) == 0 
                             else len( set(s).intersection(set(ps) ))/ len(set(s))  for s, ps in zip(test_df_preprocessed["spans"],test_df_preprocessed["predicted_span"]) ]

In [42]:
test_df_preprocessed["Fscore"] = 2 * test_df_preprocessed["Pscore"] *test_df_preprocessed["Rscore"] /(test_df_preprocessed["Pscore"] + test_df_preprocessed["Rscore"])

In [43]:
F_score= np.mean(test_df_preprocessed["Fscore"])

In [44]:
F_score

0.47098965465227693